In [11]:
output_names = ['conv2d_50/Tanh']
input_names = ['input_1']

import tensorflow as tf


def get_frozen_graph(graph_file):
    """Read Frozen Graph file from disk."""
    with tf.gfile.FastGFile(graph_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

##Input your model here
trt_graph = get_frozen_graph('trt_model/trt_graph.pb')

# Create session and load graph
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_sess = tf.Session(config=tf_config)
tf.import_graph_def(trt_graph, name='')

# Get graph input size
for node in trt_graph.node:
    if 'input_' in node.name:
        size = node.attr['shape'].shape
        image_size = [size.dim[i].size for i in range(1, 4)]
        break
print("image_size: {}".format(image_size))


# input and output tensor names.
input_tensor_name = input_names[0] + ":0"
output_tensor_name = output_names[0] + ":0"

print("input_tensor_name: {}\noutput_tensor_name: {}".format(
    input_tensor_name, output_tensor_name))

output_tensor = tf_sess.graph.get_tensor_by_name(output_tensor_name)

image_size: [256, 256, 3]
input_tensor_name: input_1:0
output_tensor_name: conv2d_50/Tanh:0


In [12]:
from tensorflow.keras.preprocessing import image
import os
##Input your testing images directory here
list_dir = os.listdir('./testingImages')
print((list_dir))

['1434.png', '1363.png', '1302.png', '1333.png', '1307.png', '1368.png', '1425.png', '1395.png', '1316.png', '1432.png', '1394.png', '1366.png', '1341.png', '1420.png', '1305.png', '1329.png', '1409.png', '1386.png', '1442.png', '1365.png', '1410.png', '1372.png', '1424.png', '1370.png', '1314.png', '1389.png', '1433.png', '.ipynb_checkpoints', '1436.png', '1422.png', '1417.png', '1362.png', '1448.png', '1391.png', '1441.png', '1403.png', '1349.png', '1380.png', '1447.png', '1301.png', '1374.png', '1306.png', '1375.png', '1323.png', '1297.png', '1299.png', '1423.png', '1308.png', '1309.png', '1344.png', '1326.png', '1426.png', '1384.png', '1340.png', '1397.png', '1358.png', '1318.png', '1407.png', '1356.png', '1418.png', '1296.png', '1399.png', '1338.png', '1313.png', '1331.png', '1361.png', '1381.png', '1346.png', '1303.png', '1332.png', '1315.png', '1393.png', '1367.png', '1347.png', '1359.png', '1322.png', '1357.png', '1379.png', '1371.png', '1383.png', '1414.png', '1312.png', '1317

In [13]:
from tensorflow.keras.preprocessing import image
import numpy as np
def load_preprocess(path):
    """Preprocesses image for the model."""
    img = image.load_img(path, target_size = (256,256))
    img1 = image.img_to_array(img)
    img1 = np.expand_dims(img1, axis=0)
    img1 = img1/127.5 - 1.
    return img1

In [ ]:
from PIL import Image
import time
time_array = []
for i in list_dir:
    path = 'testingImages/{}'.format(i)
    print(path)
    img = load_preprocess(path)
    feed_dict = {input_tensor_name:img}
    start_time = time.time()
    dehazed_image = tf_sess.run(output_tensor, feed_dict)
    time_array.append(time.time() - start_time)
    dehazed_image = dehazed_image*0.5 + 0.5
    dehazed_image = np.reshape(dehazed_image, (256,256,3))
    img = Image.fromarray((dehazed_image*255).astype(np.uint8))
    ## Replace first argument with the directory you wish to save
    img.save('output/{}'.format(i))

testingImages/1434.png


In [6]:
time_array

[]